# Example usage of the Feature Catalog API

In [ ]:
%load_ext autoreload
%autoreload 2

from pyspark.sql import SparkSession

from feature_catalog.main import compute_features
from feature_catalog.feature_groups.zone import Zone
from feature_catalog.feature_groups.zone_likelyhood import ZoneLikelyhood

spark = SparkSession.builder.getOrCreate()

# Basic usage

Define the features you want to compute and call the `compute_features` function.

In [13]:
aggregation_level = "avatarId"
scope = spark.read.parquet("data/wow.parquet").select(aggregation_level).distinct()

features = compute_features(
    scope=scope, 
    feature_groups=[
        Zone(spark=spark, features_of_interest=["darkshore_count"], aggregation_level=aggregation_level),
    ]).toPandas()
    
print("shape: ", features.shape)
print(features.head())

shape:  (91064, 2)
   avatarId  darkshore_count
0        29                3
1      1806                0
2      9233                0
3      2040                0
4        26                0


### You can compute features on different aggregation levels

As long as the feature group supports it you can compute features on different aggregation levels.
One call to the feature catalog (with `compute_features`) will always only be on one level since it makes sense to compute your features on a specific level.

See example below.


In [14]:
for aggregation_level in ["avatarId", "guild"]:
    scope = spark.read.parquet("data/wow.parquet").select(aggregation_level).distinct()

    features = compute_features(
        scope=scope, 
        feature_groups=[
            ZoneLikelyhood(spark=spark, features_of_interest=["darkshore_likelyhood"], aggregation_level=aggregation_level),
        ]).toPandas()
        
    print("shape: ", features.shape)
    print(features.head())

shape:  (91064, 2)
   avatarId  darkshore_likelyhood
0         0                   0.0
1         1                   0.0
2         2                   0.0
3         3                   0.0
4         4                   0.0


shape:  (514, 2)
   guild  darkshore_likelyhood
0     -1              0.000226
1      0              0.000084
2      1              0.000209
3      2              0.000166
4      3              0.000163
